# Split text and tag

Smoker and Family_hist can have no tag

In [ ]:
#according to flair corpus "UD_ENGLISH", train:test:dev = 6:1:1
from os import listdir
from sklearn.model_selection import train_test_split
file_path_1 = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
file_path_2 = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set2/"
names_1 = [f for f in listdir(file_path_1) if f.endswith('.xml')]
names_2 = [f for f in listdir(file_path_2) if f.endswith('.xml')]
train_n_1, test_n_1 = train_test_split(names_1, test_size=0.125, random_state=42)
train_n_1, dev_n_1 = train_test_split(train_n_1, test_size=1/7, random_state=42)
print(len(train_n_1), len(test_n_1), len(dev_n_1))

train_n_2, test_n_2 = train_test_split(names_2, test_size=0.125, random_state=42)
train_n_2, dev_n_2 = train_test_split(train_n_2, test_size=1/7, random_state=42)
print(len(train_n_2), len(test_n_2), len(dev_n_2))


In [ ]:
import pandas as pd
import numpy as np
all1_df = pd.DataFrame(np.zeros((len(names_1), 2), dtype=object), columns=['text', 'annotation'])
all2_df = pd.DataFrame(np.zeros((len(names_2), 2), dtype=object), columns=['text', 'annotation'])

In [ ]:
import re
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

train_df_1 = pd.DataFrame(np.zeros((len(train_n_1), 2), dtype=object), columns=['text', 'annotation'])
dev_df_1 = pd.DataFrame(np.zeros((len(dev_n_1), 2), dtype=object), columns=['text', 'annotation'])
test_df_1 = pd.DataFrame(np.zeros((len(test_n_1), 2), dtype=object), columns=['text', 'annotation'])
train_df_2 = pd.DataFrame(np.zeros((len(train_n_2), 2), dtype=object), columns=['text', 'annotation'])
dev_df_2 = pd.DataFrame(np.zeros((len(dev_n_2), 2), dtype=object), columns=['text', 'annotation'])
test_df_2 = pd.DataFrame(np.zeros((len(test_n_2), 2), dtype=object), columns=['text', 'annotation'])

def to_df(names,df, file_path,PHI_status = True):
    #get text
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()

        ## Get the text
        nt = re.sub('\n',' ',root[0].text)
        nt = re.sub('\t',' ',nt) 
        nt = re.sub('"',"'",nt)
        ## sample 214 has a weird character
        nt = re.sub('>','&gt;',nt) 
        nt = re.sub('<','&lt;',nt)
        df['text'][n] = nt
        n+=1
    
    #get annotations
    n = 0
    for name in names:
        tree = ET.parse(file_path + name)
        root = tree.getroot()
        ## Get the labels

        tag_list = []
        # get PHI labels if there are any
        if PHI_status == True:
            PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
            for k in range(PHI,len(root[1])):
                tag_list.append((root[1][k].attrib['text'],root[1][k].tag))
        
        # get the rest of labels
        for k in range(len(root[1])):
            for m in range(len(root[1][k])):
                if root[1][k][m].attrib.keys().__contains__('text') == False:
                    continue
                tag_list.append((root[1][k][m].attrib['text'],root[1][k][m].tag))
        
        df['annotation'][n] = tag_list
        n+=1
    return df

#train_1 = to_df(train_n_1,train_df_1, file_path_1)
#dev_1 = to_df(dev_n_1,dev_df_1, file_path_1)
#test_1 = to_df(test_n_1,test_df_1, file_path_1)
#train_2 = to_df(train_n_2,train_df_2, file_path_2, PHI_status = False)
#dev_2 = to_df(dev_n_2,dev_df_2, file_path_2, PHI_status = False)
#test_2 = to_df(test_n_2,test_df_2, file_path_2, PHI_status = False)
#all_1 = to_df(names_1,all1_df, file_path_1)
all_2 = to_df(names_2,all2_df, file_path_2, PHI_status = False)

In [ ]:
train = pd.concat([train_1, train_2], ignore_index=True)
dev = pd.concat([dev_1, dev_2], ignore_index=True)
test = pd.concat([test_1, test_2], ignore_index=True)

In [ ]:

tree = ET.parse(file_path + train_n[365])
root = tree.getroot()
PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
for k in range(PHI,len(root[1])):
    print((root[1][k].attrib['text'],root[1][k].tag))


In [ ]:
train['annotation'][365]

# Functions to convert dataframe into input format
520 files takes 3 min\
Please use create_labs function instead of mark_sentences\
source: https://medium.com/thecyphy/training-custom-ner-model-using-flair-df1f9ea9c762


In [ ]:
from tqdm import tqdm
from difflib import SequenceMatcher
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string


def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

## replace !!mark_sentence!! to better label the text with more than one word
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def to_txt(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            #text_ = text    
            print(df.index[df['text']== text].tolist())    
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)

            #for i in d.keys():
            #    f.writelines(i + ' ' + d[i] +'\n')
            #f.writelines('\n')
            for i in range(d.shape[0]):
                f.writelines(d['word'][i] + ' ' + d['label'][i] +'\n')
            f.writelines('\n')
def to_csv(df, filepath):
    for text, annotation in zip(df.text, df.annotation):
            #text = clean(text)
            #text_ = text    
            print(df.index[df['text']== text].tolist())    
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = create_labs(text, match_list)
            d.to_csv(filepath, index = False, header = False, mode = 'a')
            
def main(input,save_path):
    ## An example dataframe.
    #data = pd.DataFrame([["Horses are too tall and they pretend to care about your feelings", [("Horses", "ANIMAL")]],
    #              ["Who is Shaka Khan?", [("Shaka Khan", "PERSON")]],
    #              ["I like London and Berlin.", [("London", "LOCATION"), ("Berlin", "LOCATION")]],
    #              ["There is a banyan tree in the courtyard", [("banyan tree", "TREE")]]], columns=['text', 'annotation'])
    data = input
    ## path to save the txt file.
    ## creating the file.
    to_txt(data, save_path)
    
#if __name__ == '__main__':
path = 'C:/Users/Leste/Documents/GitHub/nlpsumm/BERT/data_processed/'
#main(train,path+'train.csv')
#main(dev,path+'dev.csv')
#main(test,path+'test.csv')
#main(all_1,path+'all_1.txt')
main(all_2,path+'all_2.txt')

# Bunch of code for testing errors
## trash

In [ ]:
def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string
text = train['text'][0]
annotation = train['annotation'][0]
#pattern = pattern.strip()
match_list = []
for i in annotation:
    a, text_ = matcher(text, i[0])
    match_list.append((a[0][0], a[0][1], i[1]))


match_list

In [ ]:
def create_labs(s,match_list):
    labs = ['O' for i in range(len(s.split()))]
    word_dict = pd.DataFrame({'word':s.split(),'label':labs})

    for start, end, e_type in match_list:
        index = len(re.findall(r' +',s[0:start]))-1
        num_words = len(s[start:end].split())
        index,num_words

        if num_words > 1:
            word_dict.loc[index,'label'] = 'B-' + e_type
            for i in range(1,num_words):
                word_dict.loc[index+i,'label'] = 'I-' + e_type
        else:
            word_dict.loc[index,'label'] = 'B-' + e_type
    return word_dict

x = create_labs(train['text'][0],match_list)

## test how to find the word using index

In [ ]:
        
a = s.split()
a.index('smoking')
len(re.findall(r' +',s[0:start]))
a[86],len(s[start:end].split())
#temp_str = pd.DataFrame(s.split(), columns=['word'])
#temp_str['len'] = temp_str['word'].apply(lambda x: len(x))
#temp_str['cumsum'] = temp_str['len'].cumsum()
#temp_str

## find which label went crazy and can't match text

In [ ]:
match_list = []
text = train['text'][176]
#text = "Horses are too tall and they pretend to care about your feelings"
annotation = train['annotation'][176]
for i in annotation:
    print(i)
    a, text_ = matcher(text, i[0])
    print(a)
    match_list.append((a[0][0], a[0][1], i[1]))
    


In [ ]:
train['text'][214]
#tree = ET.parse(file_path + names[62])
#root = tree.getroot()
#root[0].text

In [ ]:
names[214]

In [ ]:
tree = ET.parse(file_path + names[0])
root = tree.getroot()
## Get the labels
# skip FAMILY_HIST: doesn't have a text
#skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')
# get all labels except for PHI
tag_list = []
root[1][5][0].attrib['text']

# create a list of tag as tuple

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = 'C:/Users/Leste/Desktop/BDD data'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt')

In [ ]:
corpus.train[0].to_tagged_string('ner')

In [ ]:
file_path = "C:/Users/Leste/Desktop/BDD data/extracted/training-RiskFactors-Complete-Set1/"
names = [f for f in listdir(file_path) if f.endswith('.xml')]
tree = ET.parse(file_path + names[0])
root = tree.getroot()

## skip FAMILY_HIST: doesn't have a text
skip = [root[1][x].tag for x in range(len(root[1]))].index('FAMILY_HIST')
PHI = [root[1][x].tag for x in range(len(root[1]))].index('PHI')

## get all labels except for PHI
tag_list = []
for n in range(len(root[1])):
    if n == skip:
        continue
    for m in range(len(root[1][n])):
        tag_list.append((root[1][n][m].attrib['text'],root[1][n][m].tag))

## get PHI labels
for n in range(PHI,len(root[1])):
    tag_list.append((root[1][n].attrib['text'],root[1][n].tag))

## clean the text

## Use trained model from flair

In [ ]:
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
splitter = SegtokSentenceSplitter()
tagger = SequenceTagger.load('ner')
test = text_list[names[0]]
sentences = splitter.split(test)
sentences


## Try training NER with BERT

In [ ]:
# create my corpus
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# this is the folder in which train, test and dev files reside
data_folder = 'C:/Users/Leste/Documents/GitHub/nlpsumm/BERT/data_processed'

# column format indicating which columns hold the text and label(s)
columns = {0: 'text', 1: 'ner'}

corpus: Corpus = ColumnCorpus(data_folder,columns,
                              train_file = 'all_2.txt')

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer


from flair.data import Corpus
from flair.datasets import ColumnCorpus

# 1. get the corpus (create my corpus)
#file location
data_folder = 'C:/Users/Leste/Documents/GitHub/nlpsumm/BERT/data_processed'

# column format indicating which columns hold the text and label(s)
columns = {0: 'text', 1: 'ner'}

corpus: Corpus = ColumnCorpus(data_folder,columns,
                              train_file = 'all_1.txt')


In [ ]:
print(corpus.train[0].to_tagged_string('ner'))

In [ ]:

# 2. what label do we want to predict?
label_type = 'ner'


# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)


In [ ]:

# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='bert-base-uncased',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True,
                                       )


In [ ]:

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=False,
                        use_rnn=False,
                        reproject_embeddings=False,
                        )


In [ ]:

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)


In [ ]:

# 7. run fine-tuning
trainer.fine_tune('C:/Users/Leste/Documents/GitHub/nlpsumm/BERT/example-berta',
                  learning_rate=0.1,
                  mini_batch_size=32,
                  mini_batch_chunk_size=1,  # remove this parameter to speed up computation if you have a big GPU
                  max_epochs = 10)